In [3]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Lendo Dados

Lista de cidades do Brasil

In [4]:
lista_cidades = pd.read_excel('lista_cidades.xlsx', dtype=({'COD_CIDADE': str})) # , usecols=['UF', 'COD_CIDADE', 'CIDADE']

Calendario (2012-2019)

In [5]:
calendario=pd.read_excel('calendario.xlsx')
calendario.Data = calendario.Data.astype(str)

In [6]:
#Criando datas de 2020 até 2030
calendario_2020_2030 = pd.DataFrame({'Data': pd.date_range('2020-01-01', '2029-12-31'), 'chave': 'key'})

#Concatenando
calendario = pd.concat([calendario, calendario_2020_2030])
calendario.Data = pd.to_datetime(calendario.Data.astype(str), format='%Y-%m-%d')

#Criando colunas de ano, mes e dia
calendario['Ano'] = calendario.Data.dt.year
calendario['Mes'] = calendario.Data.dt.month
calendario['Dia'] = calendario.Data.dt.day

Feriados (2012-2019)

In [7]:
feriados=pd.read_excel('depara_feriados.xlsx', dtypes=({'Código IBGE\ndo Município': str}))

In [8]:
feriados_aux = feriados.copy()

In [9]:
feriados_aux = feriados_aux[['Tipo', 'UF', 'Código IBGE\ndo Município', 'Município', 'Dia', 'Mês', 'Ano', 'Data_feriados']]
feriados_aux.columns = ['Tipo', 'UF', 'COD_CIDADE', 'CIDADE', 'Dia', 'Mes', 'Ano', 'Data']

In [10]:
feriados_nac = feriados_aux[feriados_aux.Tipo == 'N'][['Tipo', 'Dia', 'Mes']].drop_duplicates()
feriados_est = feriados_aux[(feriados_aux.Tipo == 'E') & (feriados_aux.Ano == 2019)][['Tipo', 'UF', 'Dia', 'Mes', 'Data']].drop_duplicates()
feriados_mun = feriados_aux[(feriados_aux.Tipo == 'M') & (feriados_aux.Ano == 2019)][['Tipo', 'UF', 'CIDADE', 'Dia', 'Mes', 'Data']].drop_duplicates()

In [11]:
feriados_mun.isnull().sum()

Tipo      0
UF        0
CIDADE    0
Dia       0
Mes       0
Data      0
dtype: int64

In [12]:
feriados_est.head()

,Tipo,UF,Dia,Mes,Data
8918,E,AC,5,9,2019-09-05
8919,E,AC,8,3,2019-03-08
8920,E,AC,15,6,2019-06-15
8921,E,AC,17,11,2019-11-17
8922,E,AC,23,1,2019-01-23


# Criando chaves para os feriados

Estaduais

In [13]:
feriados_est['chave_est'] = 0

feriados_est['chave_est'] = feriados_est.sort_values(by=['Data']) \
                                        .groupby(['UF']) \
                                        .cumcount().apply(lambda x: f'_{x+1}')

feriados_est['chave_est'] = feriados_est.UF + feriados_est.chave_est

feriados_est.drop(['Data'], axis=1, inplace=True)

Municipais

In [15]:
feriados_mun['chave_mun'] = 0

feriados_mun['chave_mun'] = feriados_mun.sort_values(by=['Data']) \
                                        .groupby(['UF', 'CIDADE']) \
                                        .cumcount().apply(lambda x: f'_{x+1}')

feriados_mun['chave_mun'] = feriados_mun.CIDADE + '_' + feriados_mun.UF + feriados_mun.chave_mun

feriados_mun.drop(['Data'], axis=1, inplace=True)

# Calculando datas Carnaval e Páscoa

In [17]:
from datetime import date

def carnaval_pascoa(until_year):
    '''
    Função para cálculo das Datas do Carnaval e da Páscoa
    
    OBS: O Carnaval será a subtração de 47 dias da data da Páscoa
    fonte: https://www.vivaolinux.com.br/script/Calcular-a-data-do-Carnaval-e-da-Pascoa
    '''
    
    #Criando df vazio no formato padrão
    df_pascoa_carnaval = pd.DataFrame(columns=['Tipo', 'Dia', 'Mes', 'Data'])
    
    years_list = list(range(2012, until_year+1, 1))
    for ANO in years_list:
        X = 24
        Y = 5
        
        A = ANO % 19
        B = ANO % 4
        C = ANO % 7 
        D = ( 19 * A + X ) % 30
        E = ( 2 * B + 4 * C + 6 * D + Y ) % 7

        if ( (D + E ) > 9 ):
            DIA = D + E - 9
#             print ("A Pascoa de "+str(ANO)+" sera na data "+str(DIA)+" de Abril")
            MES = 4
            PASCOA = date(ANO, MES, DIA)
            CARNAVAL = date.fromordinal(PASCOA.toordinal()-47 )
#             print ("O Carnaval de "+str(ANO)+" sera na data :"+str(CARNAVAL))
        else:
            DIA = D + E + 22
#             print ("A Pascoa sera dia "+str(DIA)+" de Março")
            MES = 3
            PASCOA = date(ANO, MES, DIA)            
            CARNAVAL = date.fromordinal(PASCOA.toordinal()-47 )
#             print ("O Carnaval sera na data :"+str(CARNAVAL))
        
        carnaval = pd.to_datetime(str(CARNAVAL), format='%Y-%m-%d')
        df_pascoa_carnaval = df_pascoa_carnaval.append({'Tipo': 'N', 'Dia': carnaval.day, 'Mes': carnaval.month, 
                                                        'Data': carnaval},ignore_index=True)
        
        pascoa = pd.to_datetime(str(PASCOA), format='%Y-%m-%d')
        df_pascoa_carnaval = df_pascoa_carnaval.append({'Tipo': 'N', 'Dia': pascoa.day, 'Mes': pascoa.month, 
                                                        'Data': pascoa}, ignore_index=True)
        
    
    return df_pascoa_carnaval

In [18]:
#Criando df com datas da Pascoa e carnaval
df_pascoa_carnaval = carnaval_pascoa(2029)

In [19]:
df_pascoa_carnaval.head()

,Tipo,Dia,Mes,Data
0,N,21,2,2012-02-21
1,N,8,4,2012-04-08
2,N,12,2,2013-02-12
3,N,31,3,2013-03-31
4,N,4,3,2014-03-04


# Merge dos bases

Combinatorio Cidade x Data (Calendario completo)

In [20]:
calendario_combinado = lista_cidades.merge(calendario, on='chave', how='outer')

Combinatorio Cidade X Data e Feriados Nacionais

In [21]:
df_nac = calendario_combinado.merge(feriados_nac, on=['Dia', 'Mes'], how='outer')

#Removendo valores duplicados
df_nac.drop_duplicates(inplace=True)

In [22]:
#Duplicando as datas do dia 21-04-2019 (Feriado de Pascoa e Tiradentes no mesmo dia)
df_nac_dupli = df_nac.query("Data == '2019-04-21'")

#Concatenando com o DataFrame original
df_nac = pd.concat([df_nac, df_nac_dupli])

In [23]:
#Combinatorio feriados nacionais com a Pascoa e Carnaval
df_nac = df_nac.merge(df_pascoa_carnaval, on=['Data', 'Dia', 'Mes'], how='outer')

In [24]:
#Preenchendo o tipo de feriado
df_nac['Nac'] = np.where((df_nac.Tipo_x == 'N') | (df_nac.Tipo_y == 'N'), 1, 0)

#Dropando colunas que não interessam
df_nac.drop(['Tipo_x', 'Tipo_y', 'chave'], axis=1, inplace=True)

In [25]:
df_nac.Ano = df_nac.Ano.astype(int).astype(str)
df_nac.Mes = df_nac.Mes.astype(int).astype(str)
df_nac.Dia = df_nac.Dia.astype(int).astype(str)

### Criando chave para os feriados nacionais

In [26]:
df_nac['chave_nac'] = 0

df_nac['chave_nac'] = df_nac.query("Nac == 1").sort_values(by=['Data']) \
                                              .groupby(['Ano', 'COD_CIDADE']) \
                                              .cumcount().apply(lambda x: f'br_{x+1}')

In [27]:
df_nac.head()

,UF,COD_CIDADE,CIDADE,Data,Ano,Mes,Dia,Nac,chave_nac
0,RS,4305439,CHUI,2017-01-01,2017,1,1,1,br_1
1,RS,4317301,SANTA VITORIA DO PALMAR,2017-01-01,2017,1,1,1,br_1
2,RS,4311007,JAGUARAO,2017-01-01,2017,1,1,1,br_1
3,RS,4301305,ARROIO GRANDE,2017-01-01,2017,1,1,1,br_1
4,RS,4315602,RIO GRANDE,2017-01-01,2017,1,1,1,br_1


Combinatorio Cidade X Data e Feriados Estaduais

In [28]:
feriados_est.Mes = feriados_est.Mes.astype(str)
feriados_est.Dia = feriados_est.Dia.astype(str)

In [29]:
df_est = df_nac.merge(feriados_est, on=['Dia', 'Mes', 'UF'], how='outer')

In [30]:
df_est['Est'] = np.where(df_est.Tipo == 'E', 1, 0)
df_est.drop(['Tipo'], axis=1, inplace=True)

In [44]:
df_est.query("Nac == 1 and Est == 1")

,UF,COD_CIDADE,CIDADE,Data,Ano,Mes,Dia,Nac,chave_nac,chave_est,Est
11066733,MG,3125101,EXTREMA,2017-04-21,2017,4,21,1,br_4,MG_1,1
11066734,MG,3133600,ITAPEVA,2017-04-21,2017,4,21,1,br_4,MG_1,1
11066735,MG,3110509,CAMANDUCAIA,2017-04-21,2017,4,21,1,br_4,MG_1,1
11066736,MG,3169109,TOLEDO,2017-04-21,2017,4,21,1,br_4,MG_1,1
11066737,MG,3165404,SAPUCAI-MIRIM,2017-04-21,2017,4,21,1,br_4,MG_1,1
...,...,...,...,...,...,...,...,...,...,...,...
11099712,DF,5300108,BRASILIA,2025-04-21,2025,4,21,1,br_4,DF_1,1
11099713,DF,5300108,BRASILIA,2026-04-21,2026,4,21,1,br_4,DF_1,1
11099714,DF,5300108,BRASILIA,2027-04-21,2027,4,21,1,br_4,DF_1,1
11099715,DF,5300108,BRASILIA,2028-04-21,2028,4,21,1,br_4,DF_1,1


Combinatorio Cidade X Data e Feriados Municipais

In [33]:
feriados_mun.Mes = feriados_mun.Mes.astype(str)
feriados_mun.Dia = feriados_mun.Dia.astype(str)

In [71]:
df_mun = df_est.merge(feriados_mun, on=['Dia', 'Mes', 'UF', 'CIDADE'], how='outer')

In [73]:
df_mun['Mun'] = np.where(df_mun.Tipo == 'M', 1, 0)
df_mun.drop(['Tipo'], axis=1, inplace=True)

In [74]:
df_mun.query("Nac == 1 and Mun == 1")

,UF,COD_CIDADE,CIDADE,Data,Ano,Mes,Dia,Nac,chave_nac,chave_est,Est,chave_mun,Mun
3925391,PR,4113205,LAPA,2016-02-09,2016,2,9,1.0,br_2,NaN,0.0,LAPA_PR_1,1
3925401,PR,4113205,LAPA,2027-02-09,2027,2,9,1.0,br_2,NaN,0.0,LAPA_PR_1,1
3942077,SP,3512902,COSMORAMA,2016-02-09,2016,2,9,1.0,br_2,NaN,0.0,COSMORAMA_SP_1,1
3942087,SP,3512902,COSMORAMA,2027-02-09,2027,2,9,1.0,br_2,NaN,0.0,COSMORAMA_SP_1,1
4222375,SC,4217808,TAIO,2013-02-12,2013,2,12,1.0,br_2,NaN,0.0,TAIO_SC_1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31923147,RJ,3305752,TANGUA,2025-11-15,2025,11,15,1.0,br_9,NaN,0.0,TANGUA_RJ_3,1
31923148,RJ,3305752,TANGUA,2026-11-15,2026,11,15,1.0,br_9,NaN,0.0,TANGUA_RJ_3,1
31923149,RJ,3305752,TANGUA,2027-11-15,2027,11,15,1.0,br_9,NaN,0.0,TANGUA_RJ_3,1
31923150,RJ,3305752,TANGUA,2028-11-15,2028,11,15,1.0,br_9,NaN,0.0,TANGUA_RJ_3,1


In [75]:
df_mun['chave'] = '0'
df_mun['chave'] = np.where(df_mun.Nac == 1.0, df_mun.chave_nac, df_mun.chave)
df_mun['chave'] = np.where((df_mun.Est == 1.0) & (df_mun.chave == '0'), df_mun.chave_est, df_mun.chave)
df_mun['chave'] = np.where((df_mun.Mun == 1.0) & (df_mun.chave == '0'), df_mun.chave_mun, df_mun.chave)

In [77]:
#Removendo colunas que nao interessam
df_mun.drop(['chave_nac', 'chave_est', 'chave_mun'], axis=1, inplace=True)

In [78]:
#Removendo valores nulos
df_mun.dropna(inplace=True)

In [79]:
df_mun

,UF,COD_CIDADE,CIDADE,Data,Ano,Mes,Dia,Nac,Est,Mun,chave
0,RS,4305439,CHUI,2017-01-01,2017,1,1,1.0,0.0,0,br_1
1,RS,4305439,CHUI,2018-01-01,2018,1,1,1.0,0.0,0,br_1
2,RS,4305439,CHUI,2019-01-01,2019,1,1,1.0,0.0,0,br_1
3,RS,4305439,CHUI,2014-01-01,2014,1,1,1.0,0.0,0,br_1
4,RS,4305439,CHUI,2015-01-01,2015,1,1,1.0,0.0,0,br_1
...,...,...,...,...,...,...,...,...,...,...,...
36617175,RR,1400456,PACARAIMA,2024-02-29,2024,2,29,0.0,0.0,0,0
36617176,RR,1400456,PACARAIMA,2028-02-29,2028,2,29,1.0,0.0,0,br_2
36617177,RR,1400704,UIRAMUTA,2020-02-29,2020,2,29,0.0,0.0,0,0
36617178,RR,1400704,UIRAMUTA,2024-02-29,2024,2,29,0.0,0.0,0,0


# DataFrame Final

In [80]:
df_all = df_mun.copy()

In [81]:
#Formatando valores
df_all.Nac = df_all.Nac.astype(int).astype(str)
df_all.Est = df_all.Est.astype(int).astype(str)
df_all.Mun = df_all.Mun.astype(int).astype(str)

In [82]:
df_all.dtypes

UF                    object
COD_CIDADE            object
CIDADE                object
Data          datetime64[ns]
Ano                   object
Mes                   object
Dia                   object
Nac                   object
Est                   object
Mun                   object
chave                 object
dtype: object

In [83]:
#Dia util: Segunda a Sabado (0 a 5) e sem feriado (Nacional, Estadual ou Municipal)
df_all['dia_util'] = np.where((df_all.Nac == '0') & (df_all.Est == '0') & (df_all.Mun == '0') & (df_all.Data.dt.weekday < 6),
                              1,
                              0)

In [84]:
# df_all['dia_util'] = np.where((df_all.Nac == '0') & (df_all.Est == '0') & (df_all.Mun == '0') & (df_all.Data.dt.weekday < 5),
#                               1,
#                               0)

In [85]:
# !pip install pyarrow

In [86]:
# df_all.to_csv('dias_uteis_cidade_chave (2012-2030).csv', encoding='utf-8', index=False)

df_all.to_parquet('dias_uteis_chave.parquet', index=False, engine='pyarrow')